## COMP 569 - Harpreet Kur & Soltan Soltanli

<hr/>

### 1) The function which will generate Grid based on given inputs (Sizes)

In [98]:
import random

def generate_grid(rows, cols):
    grid = [[random.choice([0, 1]) for _ in range(cols)] for _ in range(rows)]

    r_pos = (random.randint(0, rows - 1), random.randint(0, cols - 1))
    d_pos = r_pos
    while d_pos == r_pos:
        d_pos = (random.randint(0, rows - 1), random.randint(0, cols - 1))

    grid[r_pos[0]][r_pos[1]] = 'R'
    grid[d_pos[0]][d_pos[1]] = 'D'

    return grid

grid = generate_grid(10, 10)

for row in grid:
    print(row)

[1, 0, 0, 1, 1, 0, 0, 0, 1, 0]
[1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
[1, 1, 0, 0, 0, 1, 1, 1, 0, 0]
[1, 0, 1, 0, 0, 1, 1, 1, 1, 0]
[0, 1, 0, 1, 0, 0, 0, 1, 'D', 0]
[1, 0, 0, 0, 1, 1, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 'R', 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 0]
[1, 0, 0, 1, 0, 1, 1, 1, 1, 0]


### 2) Movements

In [109]:
normal_moves = [(0, 1), (0, -1), (1, 0), (-1, 0)] # Up, Down, Left, Right
sliding_moves = [(0, 1), (0, -1), (1, 0), (-1, 0)] # Up, Down, Left, Right

### 3) Getting Start and Goal coordinates

In [102]:
start = None
goal = None

rows, cols = len(grid), len(grid[0])

for i in range(rows):
    for j in range(cols):
        if grid[i][j] == 'R':
            start = (i, j)
        if grid[i][j] == 'D':
            goal = (i, j)

if not start or not goal:
    print("Invalid grid, no start or goal point found.")

print("Start (R):", start)
print("Goal (D):", goal)

Start (R): (7, 6)
Goal (D): (4, 8)


### 4) BFS Section

In [111]:
from collections import deque

def bfs(start, goal):
    queue = deque([start])

    # We have to keep tracking the nodes we've already visited (uniquely)
    visited = set()
    visited.add(start)

    parent = {}
    node_count = 0

    while queue:
        node = queue.popleft()
        node_count += 1
        if node == goal:
            break

        # Sliding Moves
        for direction in sliding_moves:
            slide_x, slide_y = node
            while True:
                # Moving here...
                next_x = slide_x + direction[0]
                next_y = slide_y + direction[1]

                # Validating for boundary or crater ('1')
                if (0 <= next_x < len(grid) and 0 <= next_y < len(grid[0]) and grid[next_x][next_y] != '1'):
                    slide_x, slide_y = next_x, next_y

                    # We check if whether we've traversed in this node
                    if (slide_x, slide_y) not in visited:
                        visited.add((slide_x, slide_y))
                        queue.append((slide_x, slide_y))
                        parent[(slide_x, slide_y)] = node
                else:
                    break

        # Normal Moves
        for direction in normal_moves:

            # Moving here...
            next_x = node[0] + direction[0]
            next_y = node[1] + direction[1]

            # Validating for boundary or crater ('1'), also if whether we've traversed in this node
            if (0 <= next_x < len(grid) and 0 <= next_y < len(grid[0]) and grid[next_x][next_y] != '1' and (next_x, next_y) not in visited):
                visited.add((next_x, next_y))
                queue.append((next_x, next_y))
                parent[(next_x, next_y)] = node

    # Finding the whole backtrack path
    path = []

    # When we are in Goal (D)
    if node == goal:
        while node in parent:
            path.append(node)
            node = parent[node]
        path.append(start)
        path.reverse()
    return path, node_count

### 5) DFS Section

In [114]:
def dfs(start, goal):
    stack = [start]

    # We have to keep tracking the nodes we've already visited (uniquely)
    visited = set()
    visited.add(start)

    parent = {}
    node_count = 0

    while stack:
        node = stack.pop()
        node_count += 1
        if node == goal:
            break

        # Sliding Moves
        for direction in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            slide_x, slide_y = node
            while True:
                # Moving Here...
                next_x = slide_x + direction[0]
                next_y = slide_y + direction[1]

                # Validating for boundary or crater ('1')
                if (0 <= next_x < len(grid) and 0 <= next_y < len(grid[0]) and grid[next_x][next_y] != '1'):
                    slide_x, slide_y = next_x, next_y

                    # We check if whether we've traversed in this node
                    if (slide_x, slide_y) not in visited:
                        visited.add((slide_x, slide_y))
                        stack.append((slide_x, slide_y))
                        parent[(slide_x, slide_y)] = node
                else:
                    break

        # Normal Moves
        for direction in normal_moves:
            # Moving Here...
            next_x = node[0] + direction[0]
            next_y = node[1] + direction[1]

            # Validating for boundary or crater ('1'), also if whether we've traversed in this node
            if (0 <= next_x < len(grid) and 0 <= next_y < len(grid[0]) and grid[next_x][next_y] != '1' and (next_x, next_y) not in visited):
                visited.add((next_x, next_y))
                stack.append((next_x, next_y))
                parent[(next_x, next_y)] = node

    # Finding the whole backtrack path
    path = []

    # When we are in Goal (D)
    if node == goal:
        while node in parent:
            path.append(node)
            node = parent[node]
        path.append(start)
        path.reverse()
    return path, node_count

### 6) Implementation and Comparison

In [107]:
# BFS
bfs_path, bfs_nodes_explored = bfs(start, goal)

print("BFS Results:")
print("Path:", bfs_path)
print("Nodes Explored:", bfs_nodes_explored)

# DFS
dfs_path, dfs_nodes_explored = dfs(start, goal)

print("\nDFS Results:")
print("Path:", dfs_path)
print("Nodes Explored:", dfs_nodes_explored)

# Analyzing efficiency
if bfs_nodes_explored < dfs_nodes_explored:
    print("\nBFS is more efficient with fewer nodes explored.")
elif bfs_nodes_explored > dfs_nodes_explored:
    print("\nDFS is more efficient with fewer nodes explored.")
else:
    print("\nBoth BFS and DFS explored the same number of nodes.")


BFS Results:
Path: [(7, 6), (7, 8), (4, 8)]
Nodes Explored: 33

DFS Results:
Path: [(7, 6), (7, 9), (9, 9), (9, 0), (0, 0), (0, 8), (4, 8)]
Nodes Explored: 52

BFS is more efficient with fewer nodes explored.
